<a href="https://colab.research.google.com/github/rahiakela/nlp-research-and-practice/blob/main/text-similarity-works/icd10-code-highlighting/building_icd_code_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

Reference:

[Rule-based entity recognition](https://spacy.io/usage/rule-based-matching#entityruler)


In [ ]:
!pip -q install spacy
!python -m spacy download en_core_web_sm

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!pip install transformers[sentencepiece]

Just restart the colab environment.

In [1]:
import pandas as pd
import numpy as np
import re
import time
import os
import glob

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English
from spacy.tokens import Span
from spacy.language import Language
from spacy.tokens import Doc

##Define some functions

In [ ]:
@Language.component("custom_comma_remover")
def remove_comma_from_keyword(doc):
  token_list = []
  for index, token in enumerate(doc):
    # skip the loop if token contains "." or ","
    if token.text == '.' or token.text == ',':
      continue

    # replace comma with space otherwise not
    if "." in token.text:
      token_list.append(token.text.replace(".", ""))
    else:
      token_list.append(token.text)

  return Doc(doc.vocab, words=token_list)

In [ ]:
def make_icd_keyword_pattern2(icd_10_keyword_df, nlp=None):

  keywords = [row["Keyword"] for _, row in icd_10_keyword_df.iterrows()]

  phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
  patterns = list(nlp.tokenizer.pipe(keywords))
  phrase_matcher.add('keywords', patterns)

  nlp.add_pipe("custom_comma_remover")

  return phrase_matcher

In [ ]:
def make_icd_keyword_pattern(icd_10_keyword_df, nlp=None):
  keywords = []
  for _, row in icd_10_keyword_df.iterrows():
    keyword = row["Keyword"]
    keywords.append(keyword)
    # replace comma(,) with dot(.) and space and add extra two keyword
    regex = re.compile('[,]')
    if(regex.search(keyword) != None):
      keywords.append(keyword.replace(",", "."))
      keywords.append(keyword.replace(",", ""))

  phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER", )
  patterns = list(nlp.tokenizer.pipe(keywords))
  phrase_matcher.add('keywords', patterns)

  regex = re.compile('[@_!#$%^&*()<>?/\|}{~:.,]')
  reg_patterns = [{"TEXT": {"REGEX": regex}}]
  # phrase_matcher.add('reg_keywords', reg_patterns)
  # phrase_matcher.add('keywords', patterns)
  return phrase_matcher

In [ ]:
def filter_unwanted_code(code_list, page_text):
  filtered_code_list = []
  #if re.search("ICD", page_text):
  #match_list = re.findall("(ICD-[0-9][a-zA-z]*\-.+)[ ]", page_text)
  match_list = re.findall("(IC[(A-z)]-[0-9][a-zA-z]*\-.+)[ ]", page_text)
  #print("Match list:\n", match_list)
  for found_code in match_list:
    for code in code_list:
      if code in found_code:
        filtered_code_list.append(code)
  return filtered_code_list

def search_icd_code(txt_list, nlp, code_type):
  pdf_page_vocab = {}
  for txt_file in txt_list:
    with open(txt_file, "r") as f:
      page_txt = f.read()
      # filter the page that have line number instead of code
      if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
        doc = nlp(page_txt)
        code_list = [ent.text for ent in doc.ents]
        page_number = 0
        if len(code_list) != 0:
          page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
          pdf_page_vocab[page_number] = code_list
          # print(f"Page[{txt_file.split('/')[1]}]: {code_list}")

        # filter the page that dont have ICD string into it
        if code_type == "ICD-9":
          filtered_code_list = filter_unwanted_code(code_list, page_txt)
          pdf_page_vocab[page_number] = filtered_code_list
          # print(f"Page[{txt_file.split('/')[1]}]: {filtered_code_list}")

  return pdf_page_vocab

In [ ]:
def search_icd_keyword(txt_list, phrase_matcher, nlp=None):
  page_keyword_dict = {}
  # Step-4: Searching ICD-10 code
  for txt_file in txt_list:
    with open(txt_file, "r") as f:
      page_txt = f.read()
      doc = nlp(page_txt)
      matches = phrase_matcher(doc)

      keyword_list = []
      for match_id, start, end in matches:
        span = doc[start: end]
        keyword_list.append(f"{span}")

      if len(keyword_list) != 0:
        page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
        page_keyword_dict[page_number] = set(keyword_list)
        # print(f"Page[{txt_file.split('/')[1]}]: {set(keyword_list)}")
  return page_keyword_dict

##Data preprocessing

In [ ]:
columns = []
with open("icd_10_keywords.txt", "r") as f:
  txt_lines = f.readlines()
  for line in txt_lines:
    columns.append(line.strip("\n"))
print(columns[:10])

['Cholera due to Vibrio cholerae 01, biovar cholerae', 'Cholera due to Vibrio cholerae 01, biovar eltor', 'Cholera, unspecified', 'Typhoid fever, unspecified', 'Typhoid meningitis', 'Typhoid fever with heart involvement', 'Typhoid pneumonia', 'Typhoid arthritis', 'Typhoid osteomyelitis', 'Typhoid fever with other complications']


In [ ]:
data_keyword_df = pd.DataFrame(columns, columns=["Keyword"])
data_keyword_df.head()

,Keyword
0,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,"Cholera, unspecified"
3,"Typhoid fever, unspecified"
4,Typhoid meningitis


In [ ]:
data = pd.read_excel("icd10cm-order-2023_rahi.xlsx")
keyword_df = pd.DataFrame(data)
print(len(keyword_df))
keyword_df.head()

96745


,Keyword
0,Cholera
1,"Cholera due to Vibrio cholerae 01, biovar chol..."
2,"Cholera due to Vibrio cholerae 01, biovar eltor"
3,"Cholera, unspecified"
4,Typhoid and paratyphoid fevers


In [ ]:
code_df = pd.read_csv("icd_10_codes-v3.csv")
print(len(code_df))
code_df.tail()

96745


,Code
96740,U07
96741,U07.0
96742,U07.1
96743,U09
96744,U09.9


In [ ]:
icd_code_kerword_df = pd.DataFrame().assign(Code=code_df['Code'], Keyword=keyword_df['Keyword'])
icd_code_kerword_df.tail()

,Code,Keyword
96740,U07,Emergency use of U07
96741,U07.0,Vaping-related disorder
96742,U07.1,COVID-19
96743,U09,Post COVID-19 condition
96744,U09.9,"Post COVID-19 condition, unspecified"


In [ ]:
icd_code_kerword_df.shape

(96745, 2)

In [ ]:
icd_code_kerword_df.to_csv("icd_10_code_and_keywords_v2.csv", index=False)

In [ ]:
icd_code_kerword_df = pd.read_csv("icd_10_code_and_keywords_v2.csv")
icd_code_kerword_df.shape

(96746, 2)

In [ ]:
icd_code_kerword_df = pd.DataFrame().assign(Code=data_code_df['ICD-10'], Keyword=data_keyword_df['Keyword'])
icd_code_kerword_df.head()

,Code,Keyword
0,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A00.9,"Cholera, unspecified"
3,A01.00,"Typhoid fever, unspecified"
4,A01.01,Typhoid meningitis


In [ ]:
icd_code_kerword_df.to_csv("icd_10_code_keywords.csv", index=False)

In [ ]:
icd_code_kerword_df = pd.read_csv("icd_10_code_keywords.csv")
icd_code_kerword_df.head()

,Code,Keyword
0,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A00.9,"Cholera, unspecified"
3,A01.00,"Typhoid fever, unspecified"
4,A01.01,Typhoid meningitis


##Excel to CSV

In [2]:
data = pd.read_excel("WF8_CORE_Keywords_12052023.xlsx")
keyword_df = pd.DataFrame(data)
print(len(keyword_df))
keyword_df.head()

20653


,Category_0,Category_1,Category_2,SynId,Description,Short_Description,Help_Description,SynId_Reference,Keyword_Type,Keyword_Options,Keyword_Field,Keyword_Save_As
0,Keywords,Abbreviations,NaN,KW30079,ABI,ABI,cardiovascular test,ABI,LINK,NaN,NaN,NaN
1,Keywords,Abbreviations,NaN,KW30083,ABI Left,ABI Left,cardiovascular test,ABI,LINK,NaN,NaN,NaN
2,Keywords,Abbreviations,NaN,KW30082,ABI Right,ABI Right,cardiovascular test,ABI,LINK,NaN,NaN,NaN
3,Keywords,Abbreviations,NaN,KW30081,ABI Test,ABI Test,cardiovascular test,ABI,LINK,NaN,NaN,NaN
4,Keywords,Abbreviations,NaN,KW29587,BSA,BSA,body surface area,BSA,LINK,NaN,NaN,NaN


In [3]:
keyword_df1= keyword_df[["SynId", "Short_Description", "Category_1"]]
print(keyword_df1.shape)
keyword_df1.head()

(20653, 3)


,SynId,Short_Description,Category_1
0,KW30079,ABI,Abbreviations
1,KW30083,ABI Left,Abbreviations
2,KW30082,ABI Right,Abbreviations
3,KW30081,ABI Test,Abbreviations
4,KW29587,BSA,Abbreviations


In [4]:
keyword_df1.to_csv("synid_and_keywords_impairment_v19.csv", index=False)

In [5]:
core_df = pd.read_csv("synid_and_keywords_impairment_v19.csv")
core_df.head()

,SynId,Short_Description,Category_1
0,KW30079,ABI,Abbreviations
1,KW30083,ABI Left,Abbreviations
2,KW30082,ABI Right,Abbreviations
3,KW30081,ABI Test,Abbreviations
4,KW29587,BSA,Abbreviations


In [ ]:
from datetime import date

!mkdir data

In [ ]:
if not core_df.empty:
  print("Ok")

Ok


In [ ]:
core_df.to_csv(f"data/synid_and_keywords_impairment_{date.today()}.csv", index=False)

In [ ]:
core_df[46310: 46317]

,Code2,Code_Description,SynId,Field_Target
46310,Z99.8,Dependence on other enabling machines and devices,DEVR,Note
46311,Z99.81,Dependence on supplemental oxygen,O2,Note
46312,Z99.89,Dependence on other enabling machines and devices,DEVR,Note
46313,H35.461,"Secondary vitreoretinal degeneration, right eye",NaN,NaN
46314,M70.50,"Other bursitis of knee, unspecified knee",NaN,NaN
46315,H35.462,"Secondary vitreoretinal degeneration, left eye",NaN,NaN
46316,M70.51,"Other bursitis of knee, right knee",NaN,NaN


In [ ]:
# change NaN to None
core_df2 = core_df.where(pd.notnull(core_df), None)
core_df2.tail()

In [ ]:
synid_list = list(core_df2.loc[core_df2["Code"] == "S40.822D"]["SynId"])
print(synid_list)
if len(synid_list) > 0:
  synid = "No_SynId" if synid_list[0] is None else synid_list[0]
print(synid)

In [ ]:
core_df2.to_csv("icd_10_code_and_keywords_v3.csv", index=False)

##Build Code Pattern

In [ ]:
def replacer(s, newstring, index, nofail=False):
  # raise an error if index is outside of the string
  if not nofail and index not in range(len(s)):
      raise ValueError("index outside given string")

  # if not erroring, but the index is still not in the correct range..
  if index < 0:  # add it to the beginning
      return newstring + s
  if index > len(s):  # add it to the end
      return s + newstring

  # insert the new string between "slices" of the original
  return s[:index] + newstring + s[index + 1:]

In [ ]:
replacer("R10.84", "i", 1)

'Ri0.84'

In [ ]:
replacer("R10.84", "o", 2)

'R1o.84'

In [ ]:
def is_present(code_df, code_value):
  found = code_df.loc[code_df["Code"] == code_value]
  if found.size > 0:
    return True
  else:
    return False

In [ ]:
is_present(icd9_code_v1_df, "287.5")

True

In [ ]:
def make_icd_code_pattern(icd_code_df, code_type="ICD-10"):
  patterns = []
  for _, row in icd_code_df.iterrows():

    # add default pattern
    patterns.append({"label": code_type, "pattern": row["Code"]})

    # create alternate pattern
    code_patterns = []
    code_arr = row["Code"].split(".")
    if len(code_arr) > 1:
      code1 = f"{code_arr[0]}. {code_arr[1]}"
      code2 = f"{code_arr[0]} .{code_arr[1]}"
      code3 = f"{code_arr[0]} . {code_arr[1]}"
      code4 = f"{code_arr[0]} {code_arr[1]}"
      code44 = f"{code_arr[0]},{code_arr[1]}"
      code45 = f"{code_arr[0]}, {code_arr[1]}"
      code46 = f"{code_arr[0]} ,{code_arr[1]}"
      code47 = f"{code_arr[0]} , {code_arr[1]}"
      # add alternate code without dot(.)
      code48 = f"{code_arr[0]}{code_arr[1]}"
      code_patterns.extend([code1, code2, code3, code4, code44, code45, code46, code47, code48])
      # handle if the first char of code is missing
      alphabats = {"Z": "2", "B": "8", "O": "0", "S": "5", "l": "1", "G": "6", "o": "9", "i": "1"}
      for key, val in alphabats.items():
        if row["Code"].startswith(key):
          code5 = row["Code"].replace(key, val)
          code_patterns.extend([code5])
        # replcae char on 1 index if it is not present in icd9 code dataset
        # if not is_present(icd9_code_v1_df, code5) & row["Code"].find(val) == 1:
        if row["Code"].find(val) == 1:
          code6 = replacer(row["Code"], key, 1)
          code_patterns.extend([code6])
          # replcae char on 2 index
          if row["Code"].find(val) == 2:
            code7 = replacer(code6, key, 2)
            code_patterns.extend([code7])

    # handle if the "." is missing
    if code_type == "ICD-9":
      code_arr = row["Code"].split(".")
      if len(code_arr) > 1:
        code7 = row["Code"].replace(".", "")
        code_patterns.extend([code7])

    for code_pattern in code_patterns:
      #if len(code_pattern) > 1:
      patterns.append({"label": code_type, "pattern": code_pattern})
  return patterns

In [ ]:
nlp = English()

In [ ]:
icd_code_v2_df = pd.read_csv("icd_10_codes-v3.csv")
icd_code_v2_df = icd_code_v2_df.drop_duplicates()

patterns = make_icd_code_pattern(icd_code_v2_df)

# icd9_code_v1_df = pd.read_csv("icd_9_codes-v1.csv")
# patterns = make_icd_code_pattern(icd_code_v2_df, icd9_code_v1_df)

ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patterns)
# save to json file
ruler.to_disk("./icd10_code_patterns-v6.jsonl")

In [ ]:
nlp.remove_pipe("entity_ruler")

In [ ]:
icd9_code_v1_df = pd.read_csv("icd_9_codes-v1.csv")

In [ ]:
icd9_code_v1_df = pd.read_csv("icd_9_codes-v1.csv")
patterns = make_icd_code_pattern(icd9_code_v1_df, "ICD-9")

ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patterns)
# save to json file
ruler.to_disk("./icd9_code_patterns-v1.jsonl")

In [ ]:
excel_df = pd.read_excel("Synodex_CORE_KEYWORDS_impairment.xlsx")
excel_df.head()

,SynId,Category_1,Short_Description
0,KW000005,Activities,Activities of Daily Living
1,KW000003,Activities,ADL
2,KW000004,Activities,IADL
3,KW000006,Activities,Instruments of Daily Living
4,KW000001,Activities,loss of ADL


In [ ]:
keyword_df = excel_df["Short_Description"]
keyword_df.head()

0     Activities of Daily Living
1                            ADL
2                           IADL
3    Instruments of Daily Living
4                    loss of ADL
Name: Short_Description, dtype: object

In [ ]:
keyword_df.to_csv("keyword_impairment.csv", index=False)

In [ ]:
keyword_df2 = pd.read_csv("keyword_impairment.csv")
keyword_df2.head()

,Short_Description
0,Activities of Daily Living
1,ADL
2,IADL
3,Instruments of Daily Living
4,loss of ADL


##All Steps Together

##Spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

doc = nlp("""
Graham Greene is his favorite author. He wrote his first book when he was a hundred and fifty years old.
While writing this book, he had to fend off aliens and dinosaurs. Greene's second book might not have been written by him.
Greene's cat in its deathbed testimony alleged that it was the original writer of the book. The fact that plot of the book revolves around
rats conquering the world, lends credence to the idea that only a cat could have been the true writer of such an inane book.""")

matcher = Matcher(nlp.vocab)
pattern = [{"LEMMA": "write"},{"OP": "*"},{"LEMMA": "book"}]
matcher.add("testy", [pattern])

print("----- Using Matcher -----")
for sent in doc.sents:
    if matcher(sent):
        print(sent.text)

----- Using Matcher -----
He wrote his first book when he was a hundred and fifty years old.

While writing this book, he had to fend off aliens and dinosaurs.


In [ ]:
print("----- Using Dependency Matcher -----")

deppattern = [
        {"RIGHT_ID": "wrote", "RIGHT_ATTRS": {"LEMMA": "write"}},
        {"LEFT_ID": "wrote", "REL_OP": ">", "RIGHT_ID": "book",
            "RIGHT_ATTRS": {"LEMMA": "book"}}
        ]

from spacy.matcher import DependencyMatcher

dmatcher = DependencyMatcher(nlp.vocab)

dmatcher.add("BOOK", [deppattern])

for _, (start, end) in dmatcher(doc):
    print(doc[start].sent)

In [ ]:
regex = re.compile('[@_!#$%^&*()<>?/\|}{~:.,]')
True if(regex.search("Decreased white blood cell count, unspecified") == None) else False

False

In [ ]:
doc = nlp("The United States of America (USA) are commonly known as the United States (U.S. or US) or America.")

expression = r"[Uu](nited|\.?) ?[Ss](tates|\.?)"
for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)

In [ ]:
doc = nlp("The United States of America (USA) are commonly known as the United States (U.S. or US) or America.")

expression = r"[Uu](nited|\.?) ?[Ss](tates|\.?)"
for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)

Found match: United States
Found match: United States
Found match: U.S.
Found match: US


##Date Extraction

In [ ]:
!wget https://raw.githubusercontent.com/qualicen/timeline/master/history_of_germany.txt

In [ ]:
with open("history_of_germany.txt", "r") as f:
  text = f.read()

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp(text)
for ent in filter(lambda e: e.label_ == "DATE", doc.ents):
  print(ent.text)

In [ ]:
with open("page-7.txt", "r") as f:
  text = f.read()

In [ ]:
doc = nlp(text)
for ent in filter(lambda e: e.label_ == "DATE", doc.ents):
  print(ent.text)

1 year
annual


In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, ">>", ent.label_)

Pneumothorax >> PERSON
MVA >> ORG
Wisdom Teeth Extraction - 01/01/1988 >> ORG
Notes >> PRODUCT
Notes >> PRODUCT
1yr >> ORDINAL
Menopausal Status >> PERSON
Post Menopausal Bleeding: N.
Abnormal Pap: N.
Age >> WORK_OF_ART
Menarche >> ORG
15 >> CARDINAL
Obstetric History >> PERSON
HEPATITIS C AB >> ORG
QUAL >> ORG
IA >> GPE
SERUM >> ORG
0.09 NORMAL >> PERSON
TSH >> ORG
SERUM >> ORG
0.71 >> CARDINAL
SERUM >> ORG
127 >> CARDINAL
246 >> CARDINAL
CALCULATED >> ORG
172 >> CARDINAL
49 >> CARDINAL
BMP >> PRODUCT
SERUM >> ORG
3.8 >> CARDINAL
106 >> CARDINAL
NORMAL >> PERSON
22 >> CARDINAL
NORMAL >> PERSON
10 >> CARDINAL
17 >> CARDINAL
60.0 >> CARDINAL
91 >> CARDINAL
NORMAL
- CALCIUM >> PERSON
9.0 NORMAL >> QUANTITY
1 year >> DATE
Patient Portal - Epion Comprehensive Check >> ORG
Notes - Donna Tocci >> PRODUCT
annual >> DATE
1 >> CARDINAL
GAD-7 >> ORG
4 >> CARDINAL
2 >> CARDINAL
Notes >> PRODUCT


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates", use_fast=False)
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")

In [ ]:
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
nlp(text)

[{'entity_group': 'MISC',
  'score': 0.6784158,
  'word': 'RP Date',
  'start': None,
  'end': None},
 {'entity_group': 'DATE',
  'score': 0.9856512,
  'word': 'Nov 11 2021',
  'start': None,
  'end': None},
 {'entity_group': 'MISC',
  'score': 0.97850287,
  'word': 'Reviewed Surgical History Arthroscopic Surgery',
  'start': None,
  'end': None},
 {'entity_group': 'DATE',
  'score': 0.98409444,
  'word': '01/01/2015',
  'start': None,
  'end': None},
 {'entity_group': 'MISC',
  'score': 0.7657616,
  'word': 'right shoulder D&C',
  'start': None,
  'end': None},
 {'entity_group': 'DATE',
  'score': 0.95847917,
  'word': '01/01/2013',
  'start': None,
  'end': None},
 {'entity_group': 'MISC',
  'score': 0.90842485,
  'word': 'Ankle surgery',
  'start': None,
  'end': None},
 {'entity_group': 'DATE',
  'score': 0.9515049,
  'word': '01/01/2005',
  'start': None,
  'end': None},
 {'entity_group': 'MISC',
  'score': 0.9750557,
  'word': 'Left Other',
  'start': None,
  'end': None},
 {'ent

In [ ]:
for entity in nlp(text):
  if entity["entity_group"] == "DATE":
    print(entity["entity_group"], entity["word"], entity["score"])

DATE Nov 11 2021 0.9856512
DATE 01/01/2015 0.98409444
DATE 01/01/2013 0.95847917
DATE 01/01/2005 0.9515049
DATE 01/01/1999 0.9693444
DATE 01/01/1997 0.96884054
DATE 01/01/1988 0.9599809
DATE 01/01/1986 0.9692197
DATE 10/14/2021 0.9120557
DATE 11/09/21 0.9227763
DATE 11/09/21 0.90814537
DATE 11/09/21 0.8198436
DATE 11/09/21 0.8085658


In [ ]:
nlp = spacy.load('en_core_web_lg')
doc = nlp(text)
date_label = ["DATE"]

dates_pattern = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
dates = set((ent.text) for ent in doc.ents)
filtered_dates = set(date for date in dates)

sorted_dates = sorted(list(filtered_dates),reverse=True)

In [ ]:
sorted_dates

['annual',
 'about 1yr ago',
 'SERUM - 11/09/21',
 'SERUM',
 'PHQ-9',
 'PHQ-2/PHQ-9\n\n1',
 'Nov 11 2021',
 'NORMAL\n¢ LIPID PANEL',
 'NORMAL\n- POTASSIUM',
 'NORMAL\n- HEPATITIS C AB',
 'NORMAL\n- GLOMERULAR',
 'NORMAL\n- CREATININE',
 'NORMAL\n- CARBON',
 'NORMAL\n- CALCIUM',
 'NORMAL\n- BLOOD',
 'NORMAL\n- ANION',
 'N.\nAbnormal Pap: N.\nAge',
 'Menarche',
 'MVA',
 'Last Pap Smear',
 'Last Colonoscopy',
 'IA',
 'GLUCOSE',
 'Donna Tocci',
 'D&C',
 'CHOLESTEROL',
 'CALCULATED',
 'BMP',
 'Arthroscopic Surgery - 01/01/2015',
 '91',
 '9.0',
 '60.0',
 '49',
 '3.8',
 '246',
 '22',
 '2',
 '172',
 '17',
 '15',
 '138',
 '127',
 '106',
 '10',
 '1 year',
 '01/01/2013',
 '0.71',
 '0.65',
 '0.09']